In [154]:
import os
import tensorflow as tf
import cv2 as cv
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

# from keras.models import Model
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization,Conv2D,MaxPooling2D,Activation,Dropout,Lambda,Dense,Flatten, Input

In [155]:
def preProcess(img):

    return img


In [156]:
train_dir = './train/'
test_dir = './test/'
train_df = pd.read_csv('./train.csv')

train_data = []
train_jenis = []
train_warna =[]

test_data = []

input_shape = (224,224,3)

for i in os.listdir(train_dir):
    if(i is not None):
        img = cv.imread(train_dir+i)
        img = preProcess(img)

        train_data.append(img)

        cond = train_df['id'] == int(i[:-4])
        idx = train_df.loc[cond].index[0]
        train_jenis.append(train_df.iloc[idx]['jenis'])
        train_warna.append(train_df.iloc[idx]['warna'])

for i in os.listdir(test_dir):
    if(i is not None):
        img = cv.imread(test_dir+i)
        img = preProcess(img)
        test_data.append(img)


print(len(train_data))
print(len(train_jenis))
print(len(train_warna))
    


777
777
777


In [157]:
def plot_image(img):
    plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB))
    plt.show()

In [158]:
def AlexNet(input_shape, nb_class):
    inputs = Input(shape=input_shape)
    x = Conv2D(filters=96, kernel_size=(11, 11), strides=4, padding='valid', activation='relu')(inputs)


    x = MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid')(x)

    x = Conv2D(filters=256, kernel_size=(5, 5), strides=1, padding='valid', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid')(x)

    x = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='valid', activation='relu')(x)
    x = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='valid', activation='relu')(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='valid', activation='relu')(x)
    # x = Conv2D(filters=256, kernel_size=(3, 3), strides=2, padding='valid', activation='relu')(x)

    x = MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding='valid')(x)

    x = Flatten()(x)
    x = Dense(9216, activation='relu')(x)

    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)

    x = Dense(nb_class, activation='softmax')(x)
    
    model = tf.keras.models.Model(inputs=inputs, outputs=x)
    return model

In [162]:
print(len(np.unique(train_jenis)))
print(len(np.unique(train_warna)))
# color_model = AlexNet(input_shape, nb_class)
colorBranch = AlexNet(input_shape=input_shape, nb_class=len(np.unique(train_warna)))
categoryBranch = AlexNet(input_shape=input_shape, nb_class=len(np.unique(train_warna)))
model = Model(
			inputs=input_shape,
			outputs=[categoryBranch, colorBranch],
            )

2
5
